# 로지스틱 회귀분석

## 데이터셋: 유방암 진단 데이터셋(Wisconsin Diagnostic Breast Cancer, WDBC)

### WDBC 데이터셋의 반응변수
* 진단결과 (이진 반응변수): **악성(Malignant)[0]**, **양성(Benign)[1]**

### WDBC 데이터셋의 설명변수 (30개)  

1. **평균(mean)** 을 나타내는 설명변수: 세포 내 여러 영역에서 측정된 특정 지표의 평균값
* 예: mean radius(세포 핵의 평균 반지름), mean texture(세포 핵의 질감(표면의 거칠기 또는 변화 정도))  
      mean perimeter(세포 핵 둘레의 평균 길이), mean area(세포 핵의 평균 면적), mean smoothness(경계의 평활도(거칠기 정도))
* 특징: 이 값들은 전체 세포군의 대표적인 특성을 나타내어, 비교적 안정적인 패턴을 제공한다.  

2. **표준오차(error)** 를 나타내는 설명변수: 각 측정값의 변동성을 나타내는 표준오차
* 예: radius error(반지름 측정값의 변동성), texture error(질감 측정의 불확실성), perimeter error, area error 등
* 특징: 측정 과정에서 발생할 수 있는 오차와 변동 정도를 반영하여, 데이터의 신뢰도나 일관성을 평가할 수 있다.  

3. **최악(worst)** 을 나타내는 설명변수: 각 특징에 대해 관찰된 최대(또는 가장 극단적인) 값
* 예: worst radius(가장 큰 반지름 값), worst texture(가장 극단적인 질감 값), worst perimeter, worst area 등
* 특징: 악성 종양의 경우 극단적인 값들이 나타나는 경향이 있어, 진단 시 중요한 구분 요소로 작용한다.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
data = load_breast_cancer()# 30개의 독립변수
X = data.data       
y = data.target     

In [4]:
print(type(data), type(X), type(y))   # [Q] data, X, y의 type은 무엇인가?

<class 'sklearn.utils._bunch.Bunch'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [5]:
print(X.shape, y.shape)   # [Q] 이 데이터셋에는 몇 개의 샘플과 설명변수가 있는가?

(569, 30) (569,)


In [6]:
print(y)   # 이진변수인 target에는 어떤 관측값들이 있는가? 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [7]:
counts = np.bincount(y)        # 이진변수인 target에 0과 1이 몇 개씩 있는지 그 수를 반환해보자.
print("0의 개수:", counts[0])
print("1의 개수:", counts[1])

0의 개수: 212
1의 개수: 357


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
estimator = LogisticRegression(random_state=42, max_iter=10000, solver='liblinear')
selector = RFE(estimator, n_features_to_select=10)
selector = selector.fit(X_train_scaled, y_train)

#! LogisticRegression(): solver 옵션 - 최적화 알고리즘(solver)
# liblinear [<= Library for Large Linear Classification(대규모 선형 분류 문제를 효율적으로 해결하기 위해 설계된 라이브러리)]

# RFE (Recursive Feature Elimination)
# estimator: 특성의 중요도를 평가하기 위해 사용하는 기본 모델
# n_features_to_select: 최종적으로 선택할 특성의 수

In [11]:
print("선택된 변수 (True는 선택된 변수):\n", selector.support_)

# selector.support_: 각 독립변수가 최종적으로 선택되었는지를 나타내는 Boolean 배열
#                    배열의 각 원소는 해당 인덱스의 특성이 선택되었으면 True, 그렇지 않으면 False를 나타냄

선택된 변수 (True는 선택된 변수):
 [False False False False False False False  True False False  True False
 False  True False  True False False False False  True  True  True  True
 False False  True  True False False]


In [12]:
print("변수의 순위 (1-선택된 변수, 숫자가 클수록 덜 중요한 변수):\n", selector.ranking_)

# selector.ranking_: 모든 특성에 대해 중요도 순위를 매긴 배열
#                    선택된 특성은 순위 1을 가지며, 숫자가 클수록 덜 중요한 특성임

변수의 순위 (1-선택된 변수, 숫자가 클수록 덜 중요한 변수):
 [ 4  9 10  2 19  7  8  1 14 20  1 17  6  1 13  1 16 15 11 12  1  1  1  1
  3 21  1  1  5 18]


In [13]:
print(selector.n_features_)
print(selector.estimator_)

10
LogisticRegression(max_iter=10000, random_state=42, solver='liblinear')


In [19]:
selector

RFE(estimator=LogisticRegression(max_iter=10000, random_state=42,
                                 solver='liblinear'),
    n_features_to_select=10)

In [14]:
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

In [15]:
model = LogisticRegression(random_state=42, max_iter=10000)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

In [16]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[41  2]
 [ 1 70]]


In [17]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

